# Models: Movie Title Sentence Transformer

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
BASE_PATH        = '../../..'
LIB_PATH         = f'{BASE_PATH}/lib'
DATASET_PATH     = f'{BASE_PATH}/datasets'
MODEL            = 'all-roberta-large-v1'
FIELD            = 'title'

In [3]:
import sys
sys.path.append(LIB_PATH)

import torch
import pytorch_common.util as pu
from pytorch_common.util import set_device_name, \
                                get_device, \
                                LoggerBuilder

import data as dt
import data.dataset as ds

import util as ut

from sentence_transformers import SentenceTransformer

from recommender import ItemRecommenderBuilder, item_rec_sys_cfg

2023-05-04 15:05:01.976635: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Using embedded DuckDB without persistence: data will be transient


<Figure size 640x480 with 0 Axes>

## Setup

In [4]:
pu.LoggerBuilder().on_console().build()

<RootLogger root (INFO)>

In [5]:
pu.set_device_name('gpu')

pu.get_device(), torch.cuda.is_available(), torch.__version__, torch.cuda.get_arch_list()

(device(type='cuda', index=0),
 True,
 '1.11.0',
 ['sm_37',
  'sm_50',
  'sm_60',
  'sm_61',
  'sm_70',
  'sm_75',
  'sm_80',
  'sm_86',
  'compute_37'])

In [6]:
ut.set_seed(42)

In [7]:
REC_SYS_CFG = item_rec_sys_cfg(DATASET_PATH, FIELD, MODEL)

2023-05-04 15:05:02,889 - INFO - Cfg:

embedding_col: title_embedding
file_path: ../../../datasets/title-all-roberta-large-v1.json
metadata_cols:
- title
- release_year
- imdb_id
- title_tokens
name: title-all-roberta-large-v1



## Preprocesamiento

In [8]:
dataset = ds.MovieLensTMDBDataLoader.df_from_path(DATASET_PATH)

columns = ['movie_id', 'movie_release_year', 'movie_imdb_id', f'movie_{FIELD}']

movie_data = dataset \
    .pipe(dt.select, columns) \
    .pipe(dt.distinct, ['movie_id']) \
    .pipe(dt.rename, {
        'movie_id': 'id', 
        'movie_title': 'title', 
        'movie_imdb_id': 'imdb_id',  
        'movie_release_year': 'release_year',
        f'movie_{FIELD}': FIELD
    }) \
    .pipe(dt.tokenize, FIELD) \
    .pipe(dt.reset_index)

## Generacion de embeddings

In [9]:
model = SentenceTransformer(MODEL)

embeddings = model.encode(movie_data[[FIELD]].values.reshape(-1))

movie_data = movie_data \
    .pipe(dt.append_emb_vectors, embeddings, FIELD)

movie_data.to_json(REC_SYS_CFG.file_path)

del model

2023-05-04 15:05:05,908 - INFO - Load pretrained SentenceTransformer: all-roberta-large-v1
2023-05-04 15:05:07,718 - INFO - Use pytorch device: cuda


Batches:   0%|          | 0/551 [00:00<?, ?it/s]

## Evaluación

In [10]:
builder = ItemRecommenderBuilder(DATASET_PATH, [REC_SYS_CFG])

2023-05-04 15:05:17,687 - WARNING - No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction
2023-05-04 15:05:17,688 - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2023-05-04 15:05:17,802 - INFO - Use pytorch device: cuda


In [11]:
builder.item_recommender(REC_SYS_CFG.name, n_sim_items = 10) \
    .recommend(item_id=1, k=10) \
    .show()

2023-05-04 15:05:51,357 - INFO - Found 1 items by ids: [1].
2023-05-04 15:05:51,359 - INFO - Found 10 similar to 1 item.


Item Recommender: title-all-roberta-large-v1



,Similarity,Rating,.,Recommended Movies,..,Already seen movies,Rating
0,0.88,3.6,We Recommend ==>,,==> Because You Saw ==>,,3.7
1,0.85,3.9,We Recommend ==>,,==> Because You Saw ==>,,3.7
2,0.62,4.0,We Recommend ==>,,==> Because You Saw ==>,,3.7
3,0.59,4.0,We Recommend ==>,,==> Because You Saw ==>,,3.7
4,0.44,4.0,We Recommend ==>,,==> Because You Saw ==>,,3.7
5,0.38,2.0,We Recommend ==>,,==> Because You Saw ==>,,3.7
6,0.38,4.3,We Recommend ==>,,==> Because You Saw ==>,,3.7
7,0.37,3.0,We Recommend ==>,,==> Because You Saw ==>,,3.7
8,0.34,3.7,We Recommend ==>,,==> Because You Saw ==>,,3.7
